In [1]:
!pip install catboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 3.4 MB/s eta 0:00:31
    --------------------------------------- 1.3/101.7 MB 2.9 MB/s eta 0:00:35
    --------------------------------------- 2.1/101.7 MB 3.3 MB/s eta 0:00:31
   - -------------------------------------- 2.9/101.7 MB 3.4 MB/s eta 0:00:29
   - -------------------------------------- 3.7/101.7 MB 3.5 MB/s eta 0:00:28
   - -------------------------------------- 4.5/101.7 MB 3.6 MB/s eta 0:00:28
   -- ------------------------------------- 5.2/101.7 MB 3.6 MB/s eta 0:00:27
   -- ------------------------------------- 6.3/101.7 MB 3.7 MB/s eta 0:00:26
   -- ------------------------------------- 7.1/101.7 MB 3.7 MB/s eta 0:00:26
   --- ------------------------------------ 7.9/101.7 MB 3.7 MB/s eta 0:00:26
   --- ------------------------------------ 8.7/101.7 MB 3.8 MB/s eta 0:00:25


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [7]:
train_dir = r'C:\Users\mosha\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\train'  
test_dir = r'C:\Users\mosha\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [9]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [11]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [15]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [17]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

D:\college\ana\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

D:\college\ana\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 88ms/step


In [21]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step


In [23]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3232127	total: 195ms	remaining: 3m 14s
1:	learn: 1.2749162	total: 244ms	remaining: 2m 1s
2:	learn: 1.2290090	total: 291ms	remaining: 1m 36s
3:	learn: 1.1890269	total: 319ms	remaining: 1m 19s
4:	learn: 1.1562663	total: 351ms	remaining: 1m 9s
5:	learn: 1.1267079	total: 380ms	remaining: 1m 2s
6:	learn: 1.0996464	total: 408ms	remaining: 57.9s
7:	learn: 1.0739480	total: 437ms	remaining: 54.2s
8:	learn: 1.0528436	total: 463ms	remaining: 50.9s
9:	learn: 1.0338184	total: 488ms	remaining: 48.3s
10:	learn: 1.0170093	total: 515ms	remaining: 46.3s
11:	learn: 1.0021479	total: 540ms	remaining: 44.5s
12:	learn: 0.9889825	total: 567ms	remaining: 43.1s
13:	learn: 0.9775411	total: 593ms	remaining: 41.8s
14:	learn: 0.9670561	total: 620ms	remaining: 40.7s
15:	learn: 0.9560422	total: 650ms	remaining: 40s
16:	learn: 0.9456882	total: 674ms	remaining: 39s
17:	learn: 0.9370139	total: 699ms	remaining: 38.2s
18:	learn: 0.9273427	total: 724ms	remaining: 37.4s
19:	learn: 0.9209248	total: 748ms	remainin

In [25]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [27]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 57.86%


In [29]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [31]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [33]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:\Users\mosha\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\test\NonDemented\32.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
The image is classified as: NonDemented


C:\Users\mosha\AppData\Local\Temp\ipykernel_14988\3627005334.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
